# Notebook: Aplicar funciones a todo un DataFrame (pandas)
Este cuaderno demuestra técnicas para aplicar funciones sobre un DataFrame completo usando `applymap`, `apply`, operaciones vectorizadas y `np.where`. Incluye manejo de valores faltantes, medición de rendimiento y encadenamiento con `pipe`.



# 1) Importar librerías requeridas
Importar `pandas` y `numpy`; configurar opciones de visualización para ver más filas/columnas.


In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)

# 2) Crear DataFrame de ejemplo
Construir un DataFrame con números, textos y valores NaN para casos de prueba.

In [ ]:
df = pd.DataFrame({
    "numeros": [1, 2, 3, np.nan, 5, 10],
    "textos": ["a", "b", np.nan, "c", "d", "e"],
    "mixtos": ["10", 20, np.nan, "-5", "N/A", 0],
})
df

# 3) Aplicar función a todas las celdas con applymap
Definir una función segura y aplicar con `df.applymap` para transformar cada celda.

In [ ]:
def safe_transform(x):
    if pd.isna(x):
        return x
    try:
        # Si es texto con dígitos, convertir a int; si es número, multiplicar por 2
        if isinstance(x, str):
            x_clean = x.replace("N/A", "")
            return int(x_clean) if x_clean.strip() != "" and x_clean.strip().lstrip("-+").isdigit() else x.upper()
        if isinstance(x, (int, float, np.integer, np.floating)):
            return x * 2
    except Exception:
        return x
    return x

df_applymap = df.applymap(safe_transform)
df_applymap

# 4) Aplicar función a todas las columnas con apply
Usar `df.apply(func, axis=0)` para operar columna por columna (e.g., normalización numérica).

In [ ]:
def normalize_col(col):
    # Solo normaliza columnas numéricas; preserva otras
    if pd.api.types.is_numeric_dtype(col):
        return (col - col.min()) / (col.max() - col.min())
    return col

df_col_apply = df.apply(normalize_col, axis=0)
df_col_apply

# 5) Aplicar función a todas las filas con apply (axis=1)
Usar `df.apply(func, axis=1)` para crear nuevas columnas basadas en lógica fila a fila.

In [ ]:
def row_logic(row):
    # Crea una bandera si 'numeros' > 5 y 'textos' no es NaN
    cond = (pd.notna(row.get("numeros")) and row.get("numeros", 0) > 5) and pd.notna(row.get("textos"))
    return cond

df_rows = df.copy()
df_rows["flag"] = df.apply(row_logic, axis=1)
df_rows

# 6) Vectorizar operaciones para todo el DataFrame
Reemplazar `apply` por operaciones vectorizadas de pandas/numpy para mayor rendimiento.

In [ ]:
df_vec = df.copy()
# Vectorizar: multiplicar números por 2 sin apply
df_vec.loc[:, "numeros"] = df_vec["numeros"] * 2

# Vectorizar: upper() sobre textos (manejo de NaN con fillna y astype(str))
df_vec.loc[:, "textos"] = df_vec["textos"].fillna("").astype(str).str.upper().replace({"": np.nan})

# Vectorizar: convertir strings numéricos en 'mixtos' a números, dejando lo demás igual
mixtos_num = pd.to_numeric(df_vec["mixtos"], errors="coerce")
df_vec.loc[:, "mixtos"] = np.where(pd.notna(mixtos_num), mixtos_num, df_vec["mixtos"])
df_vec

# 7) Manejo de valores faltantes antes de aplicar
Detectar y tratar NaN con `fillna`, `dropna` o máscaras; evitar errores en funciones.

In [ ]:
df_na = df.copy()
# Reemplazar NaN en 'numeros' por 0 antes de cálculos
df_na["numeros_filled"] = df_na["numeros"].fillna(0)

# Eliminar filas completamente vacías (si aplica)
df_na_drop = df_na.dropna(how="all")
df_na, df_na_drop.shape

# 8) Medir rendimiento con timeit en VS Code
Comparar tiempos entre `applymap`/`apply` y vectorización usando la celda `%%timeit` y revisar el panel de salida.

In [ ]:
%%timeit
df.applymap(safe_transform)

In [ ]:
%%timeit
# Vectorizado comparable a safe_transform (parcial)
_df = df.copy()
_df.loc[:, "numeros"] = _df["numeros"] * 2
_df.loc[:, "textos"] = _df["textos"].fillna("").astype(str).str.upper().replace({"": np.nan})
mixtos_num = pd.to_numeric(_df["mixtos"], errors="coerce")
_df.loc[:, "mixtos"] = np.where(pd.notna(mixtos_num), mixtos_num, _df["mixtos"])

# 9) Usar np.where en todo el DataFrame
Aplicar condiciones globales con `np.where` para asignaciones rápidas en múltiples columnas.

In [ ]:
df_where = df.copy()
cond_numeros_altos = df_where["numeros"].fillna(-np.inf) > 5
df_where["categoria"] = np.where(cond_numeros_altos, "alto", "bajo")
df_where

# 10) Encadenar transformaciones con pipe
Definir funciones puras y encadenarlas con `df.pipe` para flujos reproducibles.

In [ ]:
def transform_numeros(df):
    df = df.copy()
    df.loc[:, "numeros"] = df["numeros"].fillna(0) * 2
    return df

def upper_textos(df):
    df = df.copy()
    df.loc[:, "textos"] = df["textos"].fillna("").astype(str).str.upper().replace({"": np.nan})
    return df

def categorize(df):
    df = df.copy()
    df.loc[:, "categoria"] = np.where(df["numeros"] > 5, "alto", "bajo")
    return df

df_piped = (
    df.pipe(transform_numeros)
      .pipe(upper_textos)
      .pipe(categorize)
)
df_piped